In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from catboost import CatBoostRegressor
import tqdm

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('data/raw/train.csv', parse_dates=['Date'])
sample_submission = pd.read_csv('data/raw/sample_submission.csv')

In [4]:
train['Date'] = pd.to_datetime(train['Date'].apply(lambda x: x[:-2] + '01'), format='%Y-%m-%d')
train['Month'] = train['Date'].dt.to_period('M').astype(str)

In [5]:
monthly_agg = (
    train
    .groupby(['Company_ID', 'Product_ID', 'Month'], as_index=False)
    .agg({'Target': 'sum'})
    .rename(columns={'Target': 'sum_sales'})
    .sort_values(by=['Company_ID', 'Product_ID', 'Month'])
)

In [6]:
d = {'Company_ID': [], 'Product_ID': [], 'Month': []}
ar = monthly_agg['Company_ID'].sort_values().unique()
br = monthly_agg['Product_ID'].sort_values().unique()
cr = monthly_agg['Month'].sort_values().unique()
for company_id in ar:
    for product_id in br:
        for month in cr:
            d['Company_ID'].append(company_id)
            d['Product_ID'].append(product_id)
            d['Month'].append(month)
d = pd.DataFrame(d)
monthly_agg = d.merge(monthly_agg, 'left', on=['Company_ID', 'Product_ID', 'Month']).fillna(0)

In [7]:
monthly_agg_saved = monthly_agg.copy()
monthly_agg

,Company_ID,Product_ID,Month,sum_sales
0,0,0,2019-01,0.0
1,0,0,2019-02,0.0
2,0,0,2019-03,0.0
3,0,0,2019-04,0.0
4,0,0,2019-05,0.0
...,...,...,...,...
3459115,3,14668,2023-08,0.0
3459116,3,14668,2023-09,0.0
3459117,3,14668,2023-10,0.0
3459118,3,14668,2023-11,0.0


In [8]:
monthly_agg['Month_date'] = pd.to_datetime(monthly_agg['Month'], format='%Y-%m')

n_lags = 15

subset = []
gb = monthly_agg.groupby(['Company_ID','Product_ID'])
for i in range(n_lags + 1):
    monthly_agg[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
    subset.append(f'prev_sum_sales_{i}')
    if i > 0:
        if i + 1 <= 9:
            feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
            monthly_agg[f'rolling_sum_sales_{i + 1}'] = monthly_agg[feats_to_mean].mean(skipna=False, axis=1)
            subset.append(f'rolling_sum_sales_{i + 1}')
            monthly_agg[f'delta_regular_rolling_sum_sales_{i + 1}'] = monthly_agg['prev_sum_sales_0'] - monthly_agg[f'rolling_sum_sales_{i + 1}']
            subset.append(f'delta_regular_rolling_sum_sales_{i + 1}')
            if i > 1:
                monthly_agg[f'delta_rolling_sum_sales_{i}'] = monthly_agg[f'rolling_sum_sales_{i}'] - monthly_agg[f'rolling_sum_sales_{i + 1}']
                subset.append(f'delta_rolling_sum_sales_{i}')
        monthly_agg[f'delta_sum_sales_{i}'] = monthly_agg['prev_sum_sales_0'] - monthly_agg[f'prev_sum_sales_{i}']
        subset.append(f'delta_sum_sales_{i}')

monthly_agg['target'] = gb['sum_sales'].shift(-1)

In [9]:
monthly_agg.dropna(subset=['target'], inplace=True)

In [10]:
monthly_agg

,Company_ID,Product_ID,Month,sum_sales,Month_date,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_regular_rolling_sum_sales_2,delta_sum_sales_1,...,delta_sum_sales_11,prev_sum_sales_12,delta_sum_sales_12,prev_sum_sales_13,delta_sum_sales_13,prev_sum_sales_14,delta_sum_sales_14,prev_sum_sales_15,delta_sum_sales_15,target
0,0,0,2019-01,0.0,2019-01-01,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,0,0,2019-02,0.0,2019-02-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,0,0,2019-03,0.0,2019-03-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,0,0,2019-04,0.0,2019-04-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
4,0,0,2019-05,0.0,2019-05-01,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459114,3,14668,2023-07,0.0,2023-07-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3459115,3,14668,2023-08,0.0,2023-08-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3459116,3,14668,2023-09,0.0,2023-09-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3459117,3,14668,2023-10,0.0,2023-10-01,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# Сортируем по Month_date (по возрастанию времени)
monthly_agg.sort_values(by=['Month_date'], inplace=True)

# Пример: берём валидацию на последние 3 месяца, которые у нас есть в данных
all_months = sorted(monthly_agg['Month_date'].unique())
N_valid_months = 3
valid_threshold = all_months[-N_valid_months]  # первая из последних 3
train_data = monthly_agg[monthly_agg['Month_date'] < valid_threshold].sample(frac=1, random_state=42)
valid_data = monthly_agg[monthly_agg['Month_date'] >= valid_threshold]

In [12]:
feature_cols = subset
cat_cols = ['Company_ID', 'Product_ID']

X_train = train_data[train_data['target'] != 0][feature_cols + cat_cols]
y_train = np.log(train_data[train_data['target'] != 0]['target'])

X_valid = valid_data[valid_data['target'] != 0][feature_cols + cat_cols]
y_valid = np.log(valid_data[valid_data['target'] != 0]['target'])

# Укажем, какие столбцы считать категориальными для CatBoost
# (CatBoost умеет работать с ними напрямую, выучивая таргет-кодирование)
cat_features_indices = [X_train.columns.get_loc(c) for c in cat_cols]  # индексы категориальных фич

In [13]:
X_train

,prev_sum_sales_0,prev_sum_sales_1,rolling_sum_sales_2,delta_regular_rolling_sum_sales_2,delta_sum_sales_1,prev_sum_sales_2,rolling_sum_sales_3,delta_regular_rolling_sum_sales_3,delta_rolling_sum_sales_2,delta_sum_sales_2,...,prev_sum_sales_12,delta_sum_sales_12,prev_sum_sales_13,delta_sum_sales_13,prev_sum_sales_14,delta_sum_sales_14,prev_sum_sales_15,delta_sum_sales_15,Company_ID,Product_ID
29490,0.0,2.0,1.0,-1.0,-2.0,0.0,0.666667,-0.666667,0.333333,0.0,...,2.0,-2.0,6.0,-6.0,3.0,-3.0,0.0,0.0,0,499
1229635,1.0,1.0,1.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,1.0,...,0.0,1.0,2.0,-1.0,0.0,1.0,2.0,-1.0,1,6182
1560952,3.0,3.0,3.0,0.0,0.0,2.0,2.666667,0.333333,0.333333,1.0,...,0.0,3.0,0.0,3.0,0.0,3.0,2.0,1.0,1,11802
1724800,1.0,1.0,1.0,0.0,0.0,0.0,0.666667,0.333333,0.333333,1.0,...,0.0,1.0,2.0,-1.0,1.0,0.0,0.0,1.0,1,14588
3174609,1.0,0.0,0.5,0.5,1.0,0.0,0.333333,0.666667,0.166667,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,9840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2970545,3.0,2.0,2.5,0.5,1.0,1.0,2.000000,1.000000,0.500000,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,6375
2099764,7.0,6.0,6.5,0.5,1.0,25.0,12.666667,-5.666667,-6.166667,-18.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,6274
1385092,1.0,0.0,0.5,0.5,1.0,0.0,0.333333,0.666667,0.166667,1.0,...,0.0,1.0,0.0,1.0,125.0,-124.0,2.0,-1.0,1,8828
2595552,7.0,3.0,5.0,2.0,4.0,0.0,3.333333,3.666667,1.666667,7.0,...,3.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,3,21


In [14]:
model = CatBoostRegressor(
    iterations=10000,
    learning_rate=0.01,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.7,
    random_seed=37,
    cat_features=cat_features_indices,
    loss_function='MAE',
    eval_metric='MAE',
    verbose=100
)

model.fit(
    X_train,
    y_train,
    eval_set=(X_valid, y_valid),
    early_stopping_rounds=100
)

0:	learn: 1.3057891	test: 1.3238891	best: 1.3238891 (0)	total: 774ms	remaining: 2h 8m 55s
100:	learn: 0.7535682	test: 0.7728051	best: 0.7728051 (100)	total: 1m	remaining: 1h 38m 59s
200:	learn: 0.6048253	test: 0.6270470	best: 0.6270470 (200)	total: 1m 54s	remaining: 1h 33m 17s
300:	learn: 0.5622744	test: 0.5858249	best: 0.5858249 (300)	total: 2m 49s	remaining: 1h 31m 13s
400:	learn: 0.5496359	test: 0.5732351	best: 0.5732351 (400)	total: 3m 42s	remaining: 1h 28m 44s
500:	learn: 0.5446383	test: 0.5684249	best: 0.5684249 (500)	total: 4m 32s	remaining: 1h 26m 8s
600:	learn: 0.5419938	test: 0.5661852	best: 0.5661852 (600)	total: 5m 19s	remaining: 1h 23m 9s
700:	learn: 0.5379765	test: 0.5644440	best: 0.5644440 (700)	total: 6m 4s	remaining: 1h 20m 34s
800:	learn: 0.5356086	test: 0.5635870	best: 0.5635870 (800)	total: 6m 49s	remaining: 1h 18m 24s
900:	learn: 0.5339863	test: 0.5630225	best: 0.5630225 (900)	total: 7m 39s	remaining: 1h 17m 19s
1000:	learn: 0.5323703	test: 0.5624260	best: 0.562426

In [15]:
from sklearn.metrics import mean_absolute_error
scale_factor = (monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12']
                .groupby(['Company_ID', 'Product_ID'])['sum_sales']
                .agg(lambda x: int(x.sum() > 0)).sum() * 3) / len(sample_submission)

y_pred_valid = np.round(np.exp(model.predict(X_valid)))
mae_valid = mean_absolute_error(np.exp(y_valid), y_pred_valid)
print("Raw MAE (valid):    ", mae_valid)
print("Scaled MAE (valid): ", mae_valid * scale_factor)

Raw MAE (valid):     23.286790955969852
Scaled MAE (valid):  9.49216526372401


In [16]:
def predict_next_month(df, model):
    df = df.copy()
    df['Month_date'] = pd.to_datetime(df['Month'], format='%Y-%m')
    df['Month_pred'] = (df['Month_date'] + pd.offsets.MonthBegin(1)).dt.to_period('M').astype(str)
    df = df.sort_values(by='Month_date')

    gb = df.groupby(['Company_ID', 'Product_ID'])
    for i in range(n_lags + 1):
        df[f'prev_sum_sales_{i}'] = gb['sum_sales'].shift(i)
        if i > 0:
            if i + 1 <= 9:
                feats_to_mean = [f'prev_sum_sales_{j}' for j in range(i + 1)]
                df[f'rolling_sum_sales_{i + 1}'] = df[feats_to_mean].mean(skipna=False, axis=1)
                df[f'delta_regular_rolling_sum_sales_{i + 1}'] = df['prev_sum_sales_0'] - df[f'rolling_sum_sales_{i + 1}']
                if i > 1:
                    df[f'delta_rolling_sum_sales_{i}'] = df[f'rolling_sum_sales_{i}'] - df[f'rolling_sum_sales_{i + 1}']
            df[f'delta_sum_sales_{i}'] = df['prev_sum_sales_0'] - df[f'prev_sum_sales_{i}']

    df['Target'] = gb['sum_sales'].shift(-1)

    to_predict = df['Target'].isna()
    df.loc[to_predict, 'Target'] = np.exp(model.predict(df.loc[to_predict, feature_cols + cat_cols])).round()
    df.loc[to_predict, 'Target'] = df.loc[to_predict, 'Target'].apply(lambda x: max(x, 0))
    return df.loc[to_predict, ['Company_ID', 'Product_ID', 'Month_pred', 'Target']]

In [17]:
mag = monthly_agg_saved[monthly_agg_saved['sum_sales'] != 0][pd.to_datetime(monthly_agg_saved['Month'], format='%Y-%m') < valid_threshold].copy()
preds = []
for i in range(N_valid_months):
    p = predict_next_month(mag, model)
    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    preds.append(p)
    mag = pd.concat([mag, p], axis=0)

preds = pd.concat(preds, axis=0)
valid_data = monthly_agg_saved[monthly_agg_saved['sum_sales'] != 0][pd.to_datetime(monthly_agg_saved['Month'], format='%Y-%m') >= valid_threshold].copy()
merged = valid_data.merge(preds, on=['Company_ID', 'Product_ID', 'Month'], suffixes=('_actual', '_pred'))
raw_mae = mean_absolute_error(merged['sum_sales_actual'], merged['sum_sales_pred'])
print("Рекурсивный прогноз - raw MAE (valid):    ", raw_mae)
print("Рекурсивный прогноз - scaled MAE (valid): ", raw_mae * scale_factor)

Рекурсивный прогноз - raw MAE (valid):     27.366341963599076
Рекурсивный прогноз - scaled MAE (valid):  11.155072464609974


In [18]:
full_train = monthly_agg.dropna(subset=['target']).sample(frac=1, random_state=42)
X_full = full_train[full_train['target'] != 0][feature_cols + cat_cols]
y_full = np.log(full_train[full_train['target'] != 0]['target'])

final_model = CatBoostRegressor(
    iterations=2900,
    learning_rate=0.01,
    depth=12,
    subsample=0.8,
    colsample_bylevel=0.7,
    random_seed=37,
    cat_features=cat_features_indices,
    loss_function='MAE',
    eval_metric='MAE',
    verbose=100
)

final_model.fit(
    X_full,
    y_full
)

0:	learn: 1.3069304	total: 676ms	remaining: 32m 38s
100:	learn: 0.7546037	total: 1m 2s	remaining: 28m 49s
200:	learn: 0.6057634	total: 1m 58s	remaining: 26m 29s
300:	learn: 0.5631164	total: 2m 49s	remaining: 24m 20s
400:	learn: 0.5502075	total: 3m 39s	remaining: 22m 50s
500:	learn: 0.5450773	total: 4m 31s	remaining: 21m 39s
600:	learn: 0.5424763	total: 5m 22s	remaining: 20m 33s
700:	learn: 0.5386592	total: 6m 14s	remaining: 19m 36s
800:	learn: 0.5363364	total: 7m 7s	remaining: 18m 41s
900:	learn: 0.5345424	total: 8m 2s	remaining: 17m 51s
1000:	learn: 0.5331433	total: 8m 55s	remaining: 16m 56s
1100:	learn: 0.5318667	total: 9m 49s	remaining: 16m 2s
1200:	learn: 0.5305208	total: 10m 44s	remaining: 15m 12s
1300:	learn: 0.5290951	total: 11m 42s	remaining: 14m 23s
1400:	learn: 0.5275390	total: 12m 38s	remaining: 13m 31s
1500:	learn: 0.5256561	total: 13m 32s	remaining: 12m 37s
1600:	learn: 0.5236967	total: 14m 28s	remaining: 11m 44s
1700:	learn: 0.5218305	total: 15m 24s	remaining: 10m 51s
180

In [19]:
sample_submission['Company_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[0]).astype(int)
sample_submission['Product_ID'] = sample_submission['Id'].apply(lambda x: x.split('_')[1]).astype(int)
sample_submission['Month_pred'] = sample_submission['Id'].apply(lambda x: x.split('_')[2]).astype(str)
sample_submission

,Id,Target,Company_ID,Product_ID,Month_pred
0,0_1_2024-01,0,0,1,2024-01
1,0_2_2024-01,0,0,2,2024-01
2,0_3_2024-01,0,0,3,2024-01
3,0_5_2024-01,0,0,5,2024-01
4,0_6_2024-01,0,0,6,2024-01
...,...,...,...,...,...
113851,3_14664_2024-03,0,3,14664,2024-03
113852,3_14665_2024-03,0,3,14665,2024-03
113853,3_14666_2024-03,0,3,14666,2024-03
113854,3_14667_2024-03,0,3,14667,2024-03


In [20]:
preds = []
mag = monthly_agg_saved.copy()
for i in range(3):
    p = predict_next_month(mag, final_model)
    preds.append(p.copy())
    p.columns = ['Company_ID', 'Product_ID', 'Month', 'sum_sales']
    mag = pd.concat([mag, p], axis=0)
preds = pd.concat(preds, axis=0)
sample_submission = sample_submission.drop(columns=['Target'], axis=1).merge(preds, 'left', on=['Company_ID', 'Product_ID', 'Month_pred'])

In [21]:
to_mult = monthly_agg_saved[monthly_agg_saved['Month'] == '2023-12'].groupby(['Company_ID', 'Product_ID'])['sum_sales'].agg(lambda x: int(x.sum() > 0)).reset_index()
for i in tqdm.tqdm(to_mult.index):
    sample_submission.loc[(sample_submission['Company_ID'] == to_mult.loc[i, 'Company_ID']) &
                          (sample_submission['Product_ID'] == to_mult.loc[i, 'Product_ID']), 'Target'] *= to_mult.loc[i, 'sum_sales']

100%|██████████| 57652/57652 [00:37<00:00, 1534.52it/s]


In [22]:
sample_submission[['Id', 'Target']]

,Id,Target
0,0_1_2024-01,6.0
1,0_2_2024-01,0.0
2,0_3_2024-01,0.0
3,0_5_2024-01,2.0
4,0_6_2024-01,3.0
...,...,...
113851,3_14664_2024-03,0.0
113852,3_14665_2024-03,186.0
113853,3_14666_2024-03,0.0
113854,3_14667_2024-03,1.0


In [23]:
formed = sample_submission[['Id', 'Target']].copy()
formed['Target'] = np.round(formed['Target'] * 0.92)
formed

,Id,Target
0,0_1_2024-01,6.0
1,0_2_2024-01,0.0
2,0_3_2024-01,0.0
3,0_5_2024-01,2.0
4,0_6_2024-01,3.0
...,...,...
113851,3_14664_2024-03,0.0
113852,3_14665_2024-03,171.0
113853,3_14666_2024-03,0.0
113854,3_14667_2024-03,1.0


In [24]:
sample_submission[['Id', 'Target']].to_csv('data/submissions/pivo.csv', index=False)